# Structuring datasets for processing

In [ ]:
import os
import re
from datetime import datetime as dt
from tqdm import tqdm
import numpy as np
import pandas as pd

RAW_DATA_LOCATION = 'data/raw/'
INPUT_DATA_LOCATION = os.path.join(RAW_DATA_LOCATION, 'corpus.tsv')
OUTPUT_DATA_LOCATION = os.path.join(RAW_DATA_LOCATION, 'corpus-localcontext.tsv')

In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations

df = pd.read_table(INPUT_DATA_LOCATION, sep='\t')

In [ ]:
df['parent_id_'] = [val.split('_')[-1] for val in tqdm(df['parent_id'].values)]

In [ ]:
data = []
for pn, parent in enumerate(df['parent_id'].unique()):
    print(parent)
    sub = df.loc[df['parent_id'].isin([parent]) | df['comment_id'].isin([parent.split('_')[-1]])]
    sub = pd.concat([sub, df.loc[df['parent_id_'].isin(sub['comment_id'].values)]], ignore_index=True).drop_duplicates()
    
    combos = list(combinations(sub.index, 2))
    for combo in tqdm(combos):
        d = sub.loc[combo[0]].rename({col:'x_'+col for col in list(df)}).to_dict()
        d.update(sub.loc[combo[1]].rename({col:'y_'+col for col in list(df)}).to_dict())
        
        d['is_parent'] = d['x_comment_id'] == d['y_parent_id_']
        d['is_child'] = d['y_comment_id'] == d['x_parent_id_']
        d['is_sibling'] = d['x_parent_id'] == d['y_parent_id']
        
        if d['is_parent'] or d['is_child'] or d['is_sibling']:
            data += [d]
    
    # x = sub.loc[combos[:,0]].copy().rename(columns={col: 'x_'+col for col in list(sub)})
    # y = sub.loc[combos[:,1]].copy().rename(columns={col: 'y_'+col for col in list(sub)})
    # 
    # sub = pd.concat([x,y], axis=1)
    # 
    # sub['context_id'] = pn
    # sub['is_parent'] = sub['x_comment_id'] == sub['y_parent_id_']
    # sub['is_child'] = sub['y_comment_id'] == sub['x_parent_id_']
    # sub['is_sibling'] = sub['x_parent_id'] == sub['y_parent_id']
    # 
    # data += [sub]
    
    # sub.columns = ['y_' + col for col in list(sub)]
    # sub[[col.replace('y_', 'x_') for col in list(sub)]] = None
    # 
    # x_columns = [col for col in list(sub) if col.startswith('x_')]
    # y_columns = [col for col in list(sub) if col.startswith('y_')]
    # for i in sub.index:
    #     sub[x_columns] = sub[y_columns].loc[i].values.tolist()
    #     sub['is_parent'] = sub['x_comment_id'] = sub['y_parent_id_']
    #     sub['is_child'] = sub['y_comment_id'] = sub['x_parent_id_']
    #     sub['is_sibling'] = sub['x_parent_id'] = sub['y_parent_id']
    #     
    #     data += [
    #         sub.loc[
    #             sub['is_parent'] | sub['is_child'] | sub['is_sibling']
    #         ].copy()
    #     ]
        
data = pd.concat(data, ignore_index=True)

In [ ]:
data.head()

In [ ]:
data.to_csv(OUTPUT_DATA_LOCATION, sep='\t', index=False, encoding='utf-8')